# Table of Contents
* [GET User data](#chapter1)
* [Get Jump Page Clicks Number](#chapter2)
* [Update Segment](#chapter3)
* [Update Post Data](#chapter4)
* [Update Optout](#chapter5)
* [Update apikey](#chapter6)
* [Update Flow](#chapter7)
* [Update Creative Date](#chapter8)
* [Update Jobs](#chapter9)
* [Update Offers](#chapter10)

In [2]:
import pandas as pd
import glob
import os
import numpy as np
from datetime import datetime
from datetime import date, timedelta
import filepath 
import infrastructure
import pygsheets 
import requests
import smartsheet
from requests.structures import CaseInsensitiveDict
from os import listdir
from os.path import isfile, join
import datetime as dt
now = datetime.now()
current_time = now.strftime("%D %H:%M:%S")
current_date = now.strftime("%y%m%d")
print("Last update time:", current_time)
import json
import time

Last update time: 06/12/24 10:07:44


In [3]:
start_time = time.time()

In [4]:
filepath.smartsheet_folder

'D:\\Users\\lilig\\Desktop\\SMS-NewData\\smartsheet'

In [5]:
offer_sms = infrastructure.get_smartsheet('offers_sms')
test_sms = infrastructure.get_smartsheet('test_sms')

In [6]:
## Enter the data foler you place output the data file 
output_filepath = filepath.output_folder
## Enter the data foler you place store the input data 
input_filepath = filepath.input_folder
Segment_filepath = filepath.output_folder+'Segment/'
smsreport_filepath = filepath.output_folder+'SMS_Report/'
smsapikey_filepath = filepath.output_folder+'SMS_Apikey/'

In [7]:
offer_sms

,Hitpath Offer ID,Email Hitpath Offer ID,Scheduling Name,Offer Name,Status,Status Changed Date,Delayed Reporting,RX Rep,Advertiser,Data Partner Relationship,...,Payout Type,Budget Status,Budget,Cap,Portal Acct Info,Live Pub IDs,Paused Pubs,Special Pub Payouts,Pitch Approval,Single/Multiple Pitch
0,NaN,NaN,NaN,EXCLUSIVE - Benefit Relief - Credit Cards - Su...,Pitch,06/10/24,NaN,Hartley Goode,Smooth Marketing,NaN,...,Variable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Single
1,NaN,NaN,Offers Approved/Declined ⬇️ drag offers here,NaN,-,NaN,NaN,NaN,Offers Ready to be Sorted,NaN,...,NaN,NaN,NaN,NaN,Offers Approved/Declined ⬇️,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,3 West Media,-,NaN,NaN,Sarah Bowe,3 West Media,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13152.0,12980.0,13152-Credit Card-3 West Media-Indigo Masterca...,Indigo Mastercard CPL v3 (non-rewards),Set Up: Pending Custom Content,03/25/24,NaN,Sarah Bowe,3 West Media,NaN,...,CPL,Non Budgeted,NaN,NaN,3 West Media(ID:28528) - Haley - SMS,NaN,NaN,NaN,NaN,Single
4,12920.0,11206.0,12920-Credit Card-3 West Media-Milestone Maste...,Milestone Mastercard - Cashback Rewards CPL (NF),Set Up: Pending Custom Content,01/24/24,NaN,Sarah Bowe,3 West Media,NaN,...,CPL,NaN,NaN,NaN,28528,NaN,NaN,NaN,Approved,Single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:

offer_sms1 = offer_sms[offer_sms['Hitpath Offer ID'].isna()== False]
#offer_sms1['LC Offer'] =  offer_sms1['Hitpath Offer ID'].astype('str').str.split(".",expand = True)[0] + '_'+offer_sms1['Abbreviation'].str.split('_',expand = True)[1] 
#offer_sms1 = offer_sms1[offer_sms1['LC Offer'].isna()== False]

In [9]:
#offer_sms1['LC Offer'].to_csv('LC_offer.csv')

# GET User data <a class="anchor" id="chapter1"></a>

In [10]:
"""
# If you want to get the jump page click data from a specific date range 
start_date = '2023-05-01'
end_date = '2023-08-22'
# start_date  
date_range = pd.date_range(start=start_date, end=end_date)

url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=person&start={first_date_of_month_url}&end={yesterday}&agents={active_pubid}"
resp = requests.get(url)
with open(output_filepath + 'User_information'+current_date+'.csv', 'w+') as f:
    f.write(resp.text)
# The code I downlodaed in the trinity, I want to combine it all in one files 
## Get the jump page clicks 

 """

'\n# If you want to get the jump page click data from a specific date range \nstart_date = \'2023-05-01\'\nend_date = \'2023-08-22\'\n# start_date  \ndate_range = pd.date_range(start=start_date, end=end_date)\n\nurl = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=person&start={first_date_of_month_url}&end={yesterday}&agents={active_pubid}"\nresp = requests.get(url)\nwith open(output_filepath + \'User_information\'+current_date+\'.csv\', \'w+\') as f:\n    f.write(resp.text)\n# The code I downlodaed in the trinity, I want to combine it all in one files \n## Get the jump page clicks \n\n '

## Get Jump Page Clicks Number <a class="anchor" id="chapter2"></a>

In [11]:
""" 
# There's maximum 45 period windows to pull the  jump page clicks from API 
# Manual download the jump page click file from API
first_date_of_month = pd.to_datetime('2023-10-01')
last_date_of_month = pd.to_datetime('2023-10-31')
first_date_of_month_url = first_date_of_month.strftime('%m/%d/%Y')
yesterday = last_date_of_month.strftime('%m/%d/%Y')
current_date = first_date_of_month.strftime("%y%m")
url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_month_url}&end={yesterday}&agents={active_pubid}"
resp = requests.get(url)
time.sleep(10)
with open(output_filepath + 'Jump Page Click from API/jumppage_click'+current_date+'.csv', 'w+') as f:
    f.write(resp.text)
"""

' \n# There\'s maximum 45 period windows to pull the  jump page clicks from API \n# Manual download the jump page click file from API\nfirst_date_of_month = pd.to_datetime(\'2023-10-01\')\nlast_date_of_month = pd.to_datetime(\'2023-10-31\')\nfirst_date_of_month_url = first_date_of_month.strftime(\'%m/%d/%Y\')\nyesterday = last_date_of_month.strftime(\'%m/%d/%Y\')\ncurrent_date = first_date_of_month.strftime("%y%m")\nurl = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_month_url}&end={yesterday}&agents={active_pubid}"\nresp = requests.get(url)\ntime.sleep(10)\nwith open(output_filepath + \'Jump Page Click from API/jumppage_click\'+current_date+\'.csv\', \'w+\') as f:\n    f.write(resp.text)\n'

In [17]:
#import new mamba 
# get active pubid 
mamba = infrastructure.get_mamba()
mamba['PUBID'] = mamba['Dataset'].str.split('_',expand = True)[2] 
active_pubid =mamba['PUBID'].unique().tolist() 
active_pubid.remove('461227') 
active_pubid.remove('461500') 
active_pubid.remove('461653') 
active_pubid.remove('461871') 
active_pubid.remove('461916')
active_pubid.remove('461794')
active_pubid.remove('461920')
middle_index = len(active_pubid) // 2
first_part = active_pubid[:middle_index]
second_part = active_pubid[middle_index:]

active_pubid_list = ['461227','461500','461653','461871','461916','461794','461920',','.join(first_part),','.join(second_part)] 
# yesterday 
yesterday = date.today() - timedelta(days=1) 
first_date_of_month = yesterday.replace(day=1) 
yesterday = yesterday.strftime('%D') 
#first_date_of_month_url = first_date_of_month.strftime('%m/%d/%y') 
first_date_of_month_url = first_date_of_month.strftime('%D') 
current_date = first_date_of_month.strftime("%y%m") 

## get last month's first date and last date 
lastMonth_last_date = first_date_of_month - timedelta(days=1) 
lastMonth_first_date = lastMonth_last_date.replace(day=1) 
#first_date_of_last_month_url = lastMonth_first_date.strftime('%m/%d/%y') 
first_date_of_last_month_url = lastMonth_first_date.strftime('%D') 
#last_date_last_month_url = lastMonth_last_date.strftime('%m/%d/%y')
last_date_last_month_url = lastMonth_last_date.strftime('%D') 
lastmonth_date = lastMonth_first_date.strftime("%y%m") 
# for i in active_pubid_list: 
#     url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_month_url}&end={yesterday}&agents={i}" 
#     resp = requests.get(url) 
#     print(url) 
#     current_month = resp.text 
#     time.sleep(25) 
#     if len(i) >10: 
#         with open(output_filepath + 'Jump Page Click from API/jumppage_click_others'+'_'+current_date+'.csv', 'w+') as f: 
#             f.write(resp.text)       
#     else: 
#         with open(output_filepath + 'Jump Page Click from API/jumppage_click_'+i+'_'+current_date+'.csv', 'w+') as f: 
#             f.write(resp.text) 
""" 
for i in active_pubid_list: 
    url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_month_url}&end={yesterday}&agents={i}" 
    resp = requests.get(url) 
    print(url) 
    current_month = resp.text 
    time.sleep(50) 
    if len(i) > 10:
        suffix = 'firsthalf' if i == ','.join(first_part) else 'secondhalf'
        with open(output_filepath + f'Jump Page Click from API/jumppage_click_{suffix}_{current_date}.csv', 'w+') as f:
            f.write(resp.text)
    else:
        with open(output_filepath + f'Jump Page Click from API/jumppage_click_{i}_{current_date}.csv', 'w+') as f:
            f.write(resp.text)
"""
def download_file(url, filepath):
    retry_count = 5
    for _ in range(retry_count):
        resp = requests.get(url)
        if len(resp.content) > 1024:  # Check if the content is greater than 1KB
            with open(filepath, 'w+') as f:
                f.write(resp.text)
            print(f"Downloaded: {filepath}")
            return True
        time.sleep(45)  # Wait for 5 seconds before retrying
    print(f"Failed to download: {filepath}")
    return False

for i in active_pubid_list:
    url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_month_url}&end={yesterday}&agents={i}"
    print(url)
   
    if len(i) > 10:
        suffix = 'firsthalf' if i == ','.join(first_part) else 'secondhalf'
        filepath = output_filepath + f'Jump Page Click from API/jumppage_click_{suffix}_{current_date}.csv'
    else:
        filepath = output_filepath + f'Jump Page Click from API/jumppage_click_{i}_{current_date}.csv'
   
    download_file(url, filepath)
    time.sleep(35)
            
# if date.today().day < 4:
#     for i in active_pubid_list: 
#         url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_last_month_url}&end={last_date_last_month_url}&agents={i}" 
#         resp = requests.get(url) 
#         print(url) 
#         time.sleep(20) 
#         if len(i) >10: 
#             with open(output_filepath + 'Jump Page Click from API/jumppage_click_others'+'_'+lastmonth_date+'.csv', 'w+') as f: 
#                 f.write(resp.text) 
#         else: 
#             with open(output_filepath + 'Jump Page Click from API/jumppage_click_'+i+"_"+lastmonth_date+'.csv', 'w+') as f: 
#                 f.write(resp.text+ '\n') 
                
""" 
# If you want to get the jump page click data from a specific date range 
start_date = '2023-05-25' 
end_date = '2023-05-31' 
# start_date date_range = pd.date_range(start=start_date, end=end_date) 
for date in date_range: 
    current_date_url = date.strftime('%m/%d/%Y') 
    current_date = date.strftime("%y%m%d") 
    url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={current_date_url}&end={current_date_url}&agents={active_pubid}" 
    resp = requests.get(url) 
    with open(output_filepath + 'Jump Page Click from API/jumppage_click'+current_date+'.csv', 'w+') as f: 
        f.write(resp.text) 
# The code I downlodaed in the trinity, I want to combine it all in one files 
## Get the jump page clicks 
os.chdir(output_filepath+'Jump Page Click from trinity') 
all_files = [i for i in glob.glob('*.{}'.format('csv'))] 
#combine all files in the list 
combined_jump_page_csv = pd.concat([pd.read_csv(f) for f in all_files ]) 
combined_jump_page_csv = combined_jump_page_csv[['date','subid_1','campaign_id','affiliate_id','hit_id']] 
combined_jump_page_csv.columns = [['date','subid_1','campaign_id','affiliate_id','hitid']] 
####Exporting master revenue file#### 
combined_jump_page_csv.to_csv(output_filepath+"jumppage_click202211-20230516.csv", index=False, encoding='utf-8-sig')


os.chdir(output_filepath+'jumppageclicks202309-202310')

all_files = [i for i in glob.glob('*.{}'.format('csv'))]
#combine all files in the list
combined_jump_page_csv = pd.concat([pd.read_csv(f) for f in all_files ])
combined_jump_page_csv = combined_jump_page_csv[['date','subid_1','campaign_id','affiliate_id','hit_id']]
combined_jump_page_csv.columns = [['date','subid_1','campaign_id','affiliate_id','hitid']]
####Exporting master revenue file####
combined_jump_page_csv.to_csv(output_filepath+"jumppage_click202309-202310.csv", index=False, encoding='utf-8-sig')

os.chdir(output_filepath+'jumppageclicks202312-202401')

all_files = [i for i in glob.glob('*.{}'.format('csv'))]
#combine all files in the list
combined_jump_page_csv = pd.concat([pd.read_csv(f) for f in all_files ])
combined_jump_page_csv = combined_jump_page_csv[['date','subid_1','campaign_id','affiliate_id','hit_id']]
combined_jump_page_csv.columns = [['date','subid_1','campaign_id','affiliate_id','hitid']]
####Exporting master revenue file####
combined_jump_page_csv.to_csv(output_filepath+"jumppage_click202312-202401.csv", index=False, encoding='utf-8-sig')
""" 

D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pygsheets\worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')
D:\Users\lilig\Desktop\github\SMS_Report\Daily_Reporting\infrastructure.py:75: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_datetime(schedule2[0],errors='coerce')
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_2036\1476750189.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start=06/01/24&end=06/11/24&agents=461227
Downloaded: D:\Users\lilig\Desktop\SMS-NewData\Jump Page Click from API/jumppage_click_461227_2406.csv
http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start=06/01/24&end=06/11/24&agents=461500
Downloaded: D:\Users\lilig\Desktop\SMS-NewData\Jump Page Click from API/jumppage_click_461500_2406.csv
http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start=06/01/24&end=06/11/24&agents=461653
Downloaded: D:\Users\lilig\Desktop\SMS-NewData\Jump Page Click from API/jumppage_click_461653_2406.csv
http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start=06/01/24&end=06/11/24&agents=461871
Downloaded: D:\Users\lilig\Desktop\SMS-NewData\Jump Page Click from API/jumppage_click_461871_2406.csv
http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3

' \n# If you want to get the jump page click data from a specific date range \nstart_date = \'2023-05-25\' \nend_date = \'2023-05-31\' \n# start_date date_range = pd.date_range(start=start_date, end=end_date) \nfor date in date_range: \n    current_date_url = date.strftime(\'%m/%d/%Y\') \n    current_date = date.strftime("%y%m%d") \n    url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={current_date_url}&end={current_date_url}&agents={active_pubid}" \n    resp = requests.get(url) \n    with open(output_filepath + \'Jump Page Click from API/jumppage_click\'+current_date+\'.csv\', \'w+\') as f: \n        f.write(resp.text) \n# The code I downlodaed in the trinity, I want to combine it all in one files \n## Get the jump page clicks \nos.chdir(output_filepath+\'Jump Page Click from trinity\') \nall_files = [i for i in glob.glob(\'*.{}\'.format(\'csv\'))] \n#combine all files in the list \ncombined_jump_page_csv = pd.concat([pd.read_csv(

In [18]:
# os.chdir(output_filepath+'jumppageclicks202402-202403')
# all_files = [i for i in glob.glob('*.{}'.format('csv'))]
# #combine all files in the list
# combined_jump_page_csv = pd.concat([pd.read_csv(f) for f in all_files ])
# combined_jump_page_csv = combined_jump_page_csv[['date','subid_1','campaign_id','affiliate_id','hit_id']]
# combined_jump_page_csv.columns = [['date','subid_1','campaign_id','affiliate_id','hitid']]
# ####Exporting master revenue file####
# combined_jump_page_csv.to_csv(output_filepath+"jumppageclicks202402-202403.csv", index=False, encoding='utf-8-sig')

In [19]:
if date.today().day < 4:
    for i in active_pubid_list: 
        url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_last_month_url}&end={last_date_last_month_url}&agents={i}" 
        resp = requests.get(url) 
        print(url) 
        time.sleep(35) 
        if len(i) > 10:
            suffix = 'firsthalf' if i == ','.join(first_part) else 'secondhalf'
            with open(output_filepath + f'Jump Page Click from API/jumppage_click_{suffix}'+'_'+lastmonth_date+'.csv', 'w+') as f:
                f.write(resp.text)
        else:
            with open(output_filepath + f'Jump Page Click from API/jumppage_click_'+i+"_"+lastmonth_date+'.csv', 'w+') as f:
                f.write(resp.text+ '\n')

In [20]:
os.chdir(output_filepath+'Jump Page Click from API') 
all_files = [i for i in glob.glob('*.{}'.format('csv'))] 
#combine all files in the list 
api_jump = pd.concat([pd.read_csv(f) for f in all_files ]) 
api_jump = api_jump.rename(columns = {'c1': 'subid_1','campaignid': 'campaign_id'}) 
api_jump['affiliate_id']= api_jump['subid_1'].astype(str).str.extract(r'(46\d{4})') 
api_jump = api_jump[['date','subid_1','campaign_id','affiliate_id','hitid']] 
####Exporting master revenue file#### 
jump_page_csv = pd.read_csv(output_filepath+"jumppage_click202211-20230516.csv")
jump_page091023 = pd.read_csv(output_filepath+"jumppage_click202309-202310.csv")
jump_page12230124 = pd.read_csv(output_filepath+"jumppage_click202312-202401.csv")
jump_page240203=pd.read_csv(output_filepath+"jumppageclicks202402-202403.csv")
jump_page = pd.concat([jump_page_csv,api_jump,jump_page091023,jump_page12230124,jump_page240203]) 
jump_page_clicks = jump_page.groupby(['affiliate_id','campaign_id','subid_1'])['hitid'].nunique().reset_index(name = 'Jump Page Clicks') 
affiliate_id = jump_page_clicks['affiliate_id'].astype(str).unique().tolist() 
jump_page_clicks =jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False)] 
jump_page_clicks['date'] = jump_page_clicks['subid_1'].str.extract(r'(\d{1,2}[A-Za-z]{3}\d{2})') 
# convert the dateinfo to datetime format using pd.to_datetime() 
jump_page_clicks['date'] = pd.to_datetime(jump_page_clicks['date'], format='%d%b%y', errors = 'coerce') 
jump_page_clicks.to_csv(output_filepath+ "jumppage_click.csv", index=False, encoding='utf-8-sig') 
duplicateRows = jump_page[jump_page['hitid'].duplicated()]
duplicateRows 
# if there's duplicates rows, please check!
"""

os.chdir(output_filepath+'Jump Page Click from API')

all_files = [i for i in glob.glob('*.{}'.format('csv'))]
#combine all files in the list
api_jump = pd.concat([pd.read_csv(f) for f in all_files ])
api_jump = api_jump.rename(columns = {'c1': 'subid_1','campaignid': 'campaign_id'})
api_jump['subid_1'] = api_jump['subid_1'].fillna('').astype(str)
api_jump['affiliate_id']= api_jump['subid_1'].astype(str).str.extract(r'(46\d{4})')
api_jump = api_jump[['date','subid_1','campaign_id','affiliate_id','hitid']]
####Exporting master revenue file####
jump_page_csv = pd.read_csv(output_filepath+"jumppage_click202211-20230516.csv")
jump_page091023 = pd.read_csv(output_filepath+"jumppage_click202309-202310.csv")
jump_page = pd.concat([jump_page_csv,api_jump,jump_page091023])
jump_page_clicks = jump_page.groupby(['affiliate_id','campaign_id','subid_1'])['hitid'].nunique().reset_index(name = 'Jump Page Clicks')
affiliate_id = jump_page_clicks['affiliate_id'].astype(str).unique().tolist()
jump_page_clicks =jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False)]
jump_page_clicks['date'] = jump_page_clicks['subid_1'].str.extract(r'(\d{1,2}[A-Za-z]{3}\d{2})')
# convert the dateinfo to datetime format using pd.to_datetime()
#added errors = coerce to passby format error
jump_page_clicks['date'] = pd.to_datetime(jump_page_clicks['date'], format='%d%b%y', errors = 'coerce') 
#jump_page_clicks['date'] = pd.to_datetime(jump_page_clicks['date']) 
#jump_page_clicks['date'] = jump_page_clicks['date'].dt.strftime('%d%b%y')
jump_page_clicks.to_csv(output_filepath+ "jumppage_click.csv", index=False, encoding='utf-8-sig')
duplicateRows = jump_page[jump_page['hitid'].duplicated()]
duplicateRows
# if there's duplicates rows, please check!
"""

D:\Users\lilig\AppData\Local\Temp\1\ipykernel_2036\4116117043.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  api_jump = pd.concat([pd.read_csv(f) for f in all_files ])
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_2036\4116117043.py:4: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  api_jump = pd.concat([pd.read_csv(f) for f in all_files ])
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_2036\4116117043.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  api_jump = pd.concat([pd.read_csv(f) for f in all_files ])
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_2036\4116117043.py:12: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  jump_page240203=pd.read_csv(output_filepath+"jumppageclicks202402-202403.csv")
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_2036\4116117043

'\n\nos.chdir(output_filepath+\'Jump Page Click from API\')\n\nall_files = [i for i in glob.glob(\'*.{}\'.format(\'csv\'))]\n#combine all files in the list\napi_jump = pd.concat([pd.read_csv(f) for f in all_files ])\napi_jump = api_jump.rename(columns = {\'c1\': \'subid_1\',\'campaignid\': \'campaign_id\'})\napi_jump[\'subid_1\'] = api_jump[\'subid_1\'].fillna(\'\').astype(str)\napi_jump[\'affiliate_id\']= api_jump[\'subid_1\'].astype(str).str.extract(r\'(46\\d{4})\')\napi_jump = api_jump[[\'date\',\'subid_1\',\'campaign_id\',\'affiliate_id\',\'hitid\']]\n####Exporting master revenue file####\njump_page_csv = pd.read_csv(output_filepath+"jumppage_click202211-20230516.csv")\njump_page091023 = pd.read_csv(output_filepath+"jumppage_click202309-202310.csv")\njump_page = pd.concat([jump_page_csv,api_jump,jump_page091023])\njump_page_clicks = jump_page.groupby([\'affiliate_id\',\'campaign_id\',\'subid_1\'])[\'hitid\'].nunique().reset_index(name = \'Jump Page Clicks\')\naffiliate_id = jump_pa

In [21]:
jump_page_clicks

,affiliate_id,campaign_id,subid_1,Jump Page Clicks,date
0,460004,64.0,460024_460001,1,NaT
151,460004,3937.0,I_MA_460398_1009733_SC.FHA_A120_5786_3_20MAR24_3,2,2024-03-20
152,460004,3937.0,I_RR2O_460724_1010851_I.R2OD_A120_5786_82_16FE...,1,2024-02-16
153,460004,3937.0,I_RR2O_460724_1018046_I.R2OD_A120_8025_82_3MAY...,4,2023-05-03
154,460004,3937.0,I_RR2O_460724_1019746_I.R2OD_A120_5786_82_28MA...,1,2024-03-28
...,...,...,...,...,...
3112859,461933,13194.0,SS_SC_SVT_61659_461933_AR0_13194_917895_9Jun24...,1,2024-06-09
3112860,461933,13194.0,SS_SC_SVT_61659_461933_AR0_13194_917895_9Jun24...,1,2024-06-09
3112861,461933,13194.0,SS_SC_SVT_61659_461933_AR0_13194_917895_9Jun24...,1,2024-06-09
3112862,461933,13194.0,SS_SC_SVT_61659_461933_AR0_13194_917895_9Jun24...,1,2024-06-09


In [22]:
""""

#import new mamba 
# get active pubid 
mamba = infrastructure.get_mamba()
mamba['PUBID'] = mamba['Dataset'].str.split('_',expand = True)[2]
active_pubid =','.join(mamba['PUBID'].unique().tolist())
active_pubid += ',460001'
# yesterday
yesterday = date.today() - timedelta(days=1)
first_date_of_month = yesterday.replace(day=1)
yesterday = yesterday.strftime('%m/%d/%Y')
first_date_of_month_url = first_date_of_month.strftime('%m/%d/%Y')
current_date =  first_date_of_month.strftime("%y%m")

## get last month's first date and last date 
lastMonth_last_date = first_date_of_month - timedelta(days=1)
lastMonth_first_date = lastMonth_last_date.replace(day=1)
first_date_of_last_month_url = lastMonth_first_date.strftime('%m/%d/%Y')
last_date_last_month_url = lastMonth_last_date.strftime('%m/%d/%Y')
lastmonth_date =  lastMonth_first_date.strftime("%y%m")

url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_month_url}&end={yesterday}&agents={active_pubid}"
resp = requests.get(url)
time.sleep(10)
with open(output_filepath + 'Jump Page Click from API/jumppage_click'+current_date+'.csv', 'w+') as f:
    f.write(resp.text)
if date.today().day < 10:
    url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={first_date_of_last_month_url}&end={last_date_last_month_url}&agents={active_pubid}"
    resp = requests.get(url)
    time.sleep(10)
    with open(output_filepath + 'Jump Page Click from API/jumppage_click'+lastmonth_date+'.csv', 'w+') as f:
        f.write(resp.text)

# If you want to get the jump page click data from a specific date range 
start_date = '2023-09-01'
end_date = '2023-09-01'
# start_date  
date_range = pd.date_range(start=start_date, end=end_date)
for date in date_range:
    current_date_url = date.strftime('%m/%d/%Y')
    current_date = date.strftime("%y%m%d")
    url = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start={current_date_url}&end={current_date_url}&agents={active_pubid}"
    resp = requests.get(url)
    with open(output_filepath + 'Jump Page Click from API/jumppage_click'+current_date+'.csv', 'w+') as f:
        f.write(resp.text)
# The code I downlodaed in the trinity, I want to combine it all in one files 
## Get the jump page clicks 

os.chdir(output_filepath+'Jump Page Click from trinity')

all_files = [i for i in glob.glob('*.{}'.format('csv'))]
#combine all files in the list
combined_jump_page_csv = pd.concat([pd.read_csv(f) for f in all_files ])
combined_jump_page_csv = combined_jump_page_csv[['date','subid_1','campaign_id','affiliate_id','hit_id']]
combined_jump_page_csv.columns = [['date','subid_1','campaign_id','affiliate_id','hitid']]
####Exporting master revenue file####
combined_jump_page_csv.to_csv(output_filepath+"jumppage_click202211-20230516.csv", index=False, encoding='utf-8-sig')

"""

'"\n\n#import new mamba \n# get active pubid \nmamba = infrastructure.get_mamba()\nmamba[\'PUBID\'] = mamba[\'Dataset\'].str.split(\'_\',expand = True)[2]\nactive_pubid =\',\'.join(mamba[\'PUBID\'].unique().tolist())\nactive_pubid += \',460001\'\n# yesterday\nyesterday = date.today() - timedelta(days=1)\nfirst_date_of_month = yesterday.replace(day=1)\nyesterday = yesterday.strftime(\'%m/%d/%Y\')\nfirst_date_of_month_url = first_date_of_month.strftime(\'%m/%d/%Y\')\ncurrent_date =  first_date_of_month.strftime("%y%m")\n\n## get last month\'s first date and last date \nlastMonth_last_date = first_date_of_month - timedelta(days=1)\nlastMonth_first_date = lastMonth_last_date.replace(day=1)\nfirst_date_of_last_month_url = lastMonth_first_date.strftime(\'%m/%d/%Y\')\nlast_date_last_month_url = lastMonth_last_date.strftime(\'%m/%d/%Y\')\nlastmonth_date =  lastMonth_first_date.strftime("%y%m")\n\nurl = f"http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&

In [23]:
print(url)

http://reporting.rxmgtracker.com/api.php?key=7e6ae66cfaf91c3e7cd446e2affedb07&type=clicks&start=06/01/24&end=06/11/24&agents=461838,461810,461452,461242,461247,461843,461768,461039,461896,461933


In [24]:
"""

os.chdir(output_filepath+'Jump Page Click from API')

all_files = [i for i in glob.glob('*.{}'.format('csv'))]
#combine all files in the list
api_jump = pd.concat([pd.read_csv(f) for f in all_files ])
api_jump = api_jump.rename(columns = {'c1': 'subid_1','campaignid': 'campaign_id'})
api_jump['subid_1'] = api_jump['subid_1'].fillna('').astype(str)
api_jump['affiliate_id']= api_jump['subid_1'].astype(str).str.extract(r'(46\d{4})')
api_jump = api_jump[['date','subid_1','campaign_id','affiliate_id','hitid']]
####Exporting master revenue file####
jump_page_csv = pd.read_csv(output_filepath+"jumppage_click202211-20230516.csv")
jump_page091023 = pd.read_csv(output_filepath+"jumppage_click202309-202310.csv")
jump_page = pd.concat([jump_page_csv,api_jump,jump_page091023])
jump_page_clicks = jump_page.groupby(['affiliate_id','campaign_id','subid_1'])['hitid'].nunique().reset_index(name = 'Jump Page Clicks')
affiliate_id = jump_page_clicks['affiliate_id'].astype(str).unique().tolist()
jump_page_clicks =jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False)]
jump_page_clicks['date'] = jump_page_clicks['subid_1'].str.extract(r'(\d{1,2}[A-Za-z]{3}\d{2})')
# convert the dateinfo to datetime format using pd.to_datetime()
#added errors = coerce to passby format error
jump_page_clicks['date'] = pd.to_datetime(jump_page_clicks['date'], format='%d%b%y', errors = 'coerce') 
#jump_page_clicks['date'] = pd.to_datetime(jump_page_clicks['date']) 
#jump_page_clicks['date'] = jump_page_clicks['date'].dt.strftime('%d%b%y')
jump_page_clicks.to_csv(output_filepath+ "jumppage_click.csv", index=False, encoding='utf-8-sig')
duplicateRows = jump_page[jump_page['hitid'].duplicated()]
duplicateRows
# if there's duplicates rows, please check!
"""

'\n\nos.chdir(output_filepath+\'Jump Page Click from API\')\n\nall_files = [i for i in glob.glob(\'*.{}\'.format(\'csv\'))]\n#combine all files in the list\napi_jump = pd.concat([pd.read_csv(f) for f in all_files ])\napi_jump = api_jump.rename(columns = {\'c1\': \'subid_1\',\'campaignid\': \'campaign_id\'})\napi_jump[\'subid_1\'] = api_jump[\'subid_1\'].fillna(\'\').astype(str)\napi_jump[\'affiliate_id\']= api_jump[\'subid_1\'].astype(str).str.extract(r\'(46\\d{4})\')\napi_jump = api_jump[[\'date\',\'subid_1\',\'campaign_id\',\'affiliate_id\',\'hitid\']]\n####Exporting master revenue file####\njump_page_csv = pd.read_csv(output_filepath+"jumppage_click202211-20230516.csv")\njump_page091023 = pd.read_csv(output_filepath+"jumppage_click202309-202310.csv")\njump_page = pd.concat([jump_page_csv,api_jump,jump_page091023])\njump_page_clicks = jump_page.groupby([\'affiliate_id\',\'campaign_id\',\'subid_1\'])[\'hitid\'].nunique().reset_index(name = \'Jump Page Clicks\')\naffiliate_id = jump_pa

In [25]:
"""
jump_page_clicks['Date'] = jump_page_clicks['subid_1'].str.extract(r'(\d{1,2}[A-Za-z]{3}\d{2})')
# convert the dateinfo to datetime format using pd.to_datetime()
# added errors = coerce to bypass format errors
jump_page_clicks['Date'] = pd.to_datetime(jump_page_clicks['Date'], format='%d%b%y', errors = 'coerce') 
jump_page_clicks
"""

"\njump_page_clicks['Date'] = jump_page_clicks['subid_1'].str.extract(r'(\\d{1,2}[A-Za-z]{3}\\d{2})')\n# convert the dateinfo to datetime format using pd.to_datetime()\n# added errors = coerce to bypass format errors\njump_page_clicks['Date'] = pd.to_datetime(jump_page_clicks['Date'], format='%d%b%y', errors = 'coerce') \njump_page_clicks\n"

In [26]:

useful_click = jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False)]['Jump Page Clicks'].sum()
useless_clicks = jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False) == False]['Jump Page Clicks'].sum()
useful_click+useless_clicks -  len(jump_page) 
# the missing 502 clicks is from problematic subid_1 
# useless clicks is also from porblematic clicks 

D:\Users\lilig\AppData\Local\Temp\1\ipykernel_2036\322886987.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  useful_click = jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False)]['Jump Page Clicks'].sum()
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_2036\322886987.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  useless_clicks = jump_page_clicks.loc[jump_page_clicks['subid_1'].str.contains(f'({"|".join(affiliate_id)})', na=False) == False]['Jump Page Clicks'].sum()


-4322172

## Update Segment <a class="anchor" id="chapter3"></a>

In [27]:

from os import listdir
from os.path import isfile, join
import datetime as dt

mamba = infrastructure.get_mamba()
current_active_segment = mamba.loc[mamba['Date'] == pd.to_datetime(date.today()),'Segment '].unique().tolist()
secondary_segment =  mamba.loc[(mamba['Date'] == pd.to_datetime(date.today())) & (mamba['Segment '].str.contains('TMO|VZN',na = False)),'Segment '].unique().tolist()

                                                                 
segment_files = [f for f in listdir(Segment_filepath) if isfile(join(Segment_filepath, f)) & ('SMS_SC_SS_Segments' in f)]
output_segment = pd.DataFrame()
segment_files.sort()
for i in segment_files: 
    segment_report = pd.read_csv(Segment_filepath+i)
    segment_report['Date'] = dt.datetime.strptime((i.split('.csv')[0][-8:]), "%Y%m%d")
    updated_segment_report = segment_report.drop(['id','next_count'],axis = 1)
    output_segment = pd.concat([updated_segment_report,output_segment], axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
    print(output_segment)
""" 
most_recent_segment_date = max([dt.datetime.strptime((date.split('.csv')[0][-8:]), "%Y%m%d") for date in segment_files])
segment_year = most_recent_segment_date.year
segment_month = most_recent_segment_date.month
segement_day = most_recent_segment_date.day

most_recent_segment_file = "SMS_SC_SS_Segments{0}{1:0>2}{2:0>2}.csv".format(segment_year,segment_month,segement_day)

segment_report = pd.read_csv( Segment_filepath+ most_recent_segment_file)

segment_report['Date'] = pd.to_datetime(dt.datetime.today().strftime("%D"))
updated_segment_report = segment_report.drop(['id','next_count'],axis = 1)

legacy_segment = pd.read_csv(output_filepath+'SMS_SC_SS_Segments.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_segment.to_csv(input_filepath+'SMS_SC_SS_Segments_backup.csv', index = False)
# This is the job stat raw data downloaded by the outsource 
legacy_segment['Date'] = pd.to_datetime(legacy_segment['Date'])
min_date = updated_segment_report['Date'].min()
legacy_segment = legacy_segment[legacy_segment['Date']<min_date]
full_segment = [legacy_segment, updated_segment_report]
output_segment = pd.concat(full_segment, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
print(output_segment)
"""
## rename on 5/24
output_segment['name'] = output_segment['name'].str.replace('FLC_I.CC_OI.4P_30DC','FLC_I.CC_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_SPK.CR_OI.4P_30DC','FLC_SPK.CR_30DC')
output_segment['name'] = output_segment['name'].str.replace('MBC_SPK.SWP_OI.4P_30DC','MBC_SPK.SWP_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_AI.CC_OI.4P_30DC','FLC_AI.CC_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_EDM.247L_OI.4P_30DC','FLC_EDM.247L_30DC')
output_segment['name'] = output_segment['name'].str.replace('MBC_NPD.RTO_OI.4P_30DC','MBC_NPD.RTO_30DC')
output_segment['name'] = output_segment['name'].str.replace('MBC_PN.FC_OI.4P_30DC','MBC_PN.FC_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_WWM.YFA.2_OI.4P_30DC','FLC_WWM.YFA.2_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_ZM.PL.2_OI.4P_30DC','FLC_ZM.PL.2_30DC')
output_segment['name'] = output_segment['name'].str.replace('FLC_SPK.SWP_OI.4P.VZN_30DC','FLC_SPK.SWP_30DC_VZN')
output_segment['name'] = output_segment['name'].str.replace('MBC_EDM.247L_OI.4P.TMO_30DC','MBC_EDM.247L_30DC_TMO')
output_segment['name'] = output_segment['name'].str.replace('MBC_WWM.YFA.2_OI.4P_30DC','MBC_WWM.YFA.2_30DC_TMO')
output_segment['Secondary Segment'] = False
output_segment['Active Segment'] = False
output_segment.loc[output_segment['name'].isin(secondary_segment),'Secondary Segment'] = True
output_segment.loc[output_segment['name'].isin(current_active_segment),'Active Segment'] = True

output_segment.to_csv(output_filepath+'SMS_SC_SS_Segments.csv', index = False)

D:\Users\lilig\AppData\Local\anaconda3\Lib\site-packages\pygsheets\worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')
D:\Users\lilig\Desktop\github\SMS_Report\Daily_Reporting\infrastructure.py:75: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_datetime(schedule2[0],errors='coerce')


                             name channel  status  count  \
0                          461243     sms  active  11531   
1          526 rto-JULY22-CLK-hzb     sms  active   1283   
2                   ARM CR 460939     sms  active  62935   
3               ARM.CR Last Month     sms  active  26541   
4                   ARMSMPL461149     sms  active   8279   
..                            ...     ...     ...    ...   
304                RTO-JULY22-CLK     sms  active   7579   
305               test-2565371505     sms  active   4255   
306  viewrenttoownhomes URL check     sms  active   6386   
307                      ZP461235     sms  active  85116   
308                 ZP Last Month     sms  active   4110   

                last_counted       Date  
0    2023-05-01 00:06:08 EST 2023-05-17  
1    2023-05-17 06:18:34 EST 2023-05-17  
2    2023-05-01 19:42:33 EST 2023-05-17  
3    2023-04-29 06:55:26 EST 2023-05-17  
4    2023-04-29 04:34:56 EST 2023-05-17  
..                       ..

                              name channel  status  count  \
0                           461243     sms  active  11531   
1           526 rto-JULY22-CLK-hzb     sms  active    945   
2                    ARMSMPL461149     sms  active   8279   
3                ARMSYF Last Month     sms  active      0   
4                   ARMUSACF461148     sms  active   2690   
...                            ...     ...     ...    ...   
3160                RTO-JULY22-CLK     sms  active   7579   
3161               test-2565371505     sms  active   4255   
3162  viewrenttoownhomes URL check     sms  active   6386   
3163                      ZP461235     sms  active  85116   
3164                 ZP Last Month     sms  active   4110   

                 last_counted       Date  
0     2023-05-01 00:06:08 EST 2023-05-30  
1     2023-05-30 07:06:31 EST 2023-05-30  
2     2023-04-29 04:34:56 EST 2023-05-30  
3     2023-04-29 05:14:10 EST 2023-05-30  
4     2023-04-29 03:27:33 EST 2023-05-30  
...      

                              name channel  status  count  \
0                           461243     sms  active  11531   
1           526 rto-JULY22-CLK-hzb     sms  active    940   
2                    ARMSMPL461149     sms  active   8279   
3                   ARMUSACF461148     sms  active   2690   
4                     B2CGC 461122     sms  active   7417   
...                            ...     ...     ...    ...   
6568                RTO-JULY22-CLK     sms  active   7579   
6569               test-2565371505     sms  active   4255   
6570  viewrenttoownhomes URL check     sms  active   6386   
6571                      ZP461235     sms  active  85116   
6572                 ZP Last Month     sms  active   4110   

                 last_counted       Date  
0     2023-05-01 00:06:08 EST 2023-06-15  
1     2023-06-15 04:47:06 EST 2023-06-15  
2     2023-04-29 04:34:56 EST 2023-06-15  
3     2023-04-29 03:27:33 EST 2023-06-15  
4     2023-05-01 13:44:37 EST 2023-06-15  
...      

                              name channel  status  count  \
0           526 rto-JULY22-CLK-hzb     sms  active    945   
1                    ARMSMPL461149     sms  active   8279   
2                   ARMUSACF461148     sms  active   2690   
3                     B2CGC 461122     sms  active   7417   
4                B2USRA Last Month     sms  active     36   
...                            ...     ...     ...    ...   
8816                RTO-JULY22-CLK     sms  active   7579   
8817               test-2565371505     sms  active   4255   
8818  viewrenttoownhomes URL check     sms  active   6386   
8819                      ZP461235     sms  active  85116   
8820                 ZP Last Month     sms  active   4110   

                 last_counted       Date  
0     2023-06-29 07:11:36 EST 2023-06-29  
1     2023-04-29 04:34:56 EST 2023-06-29  
2     2023-04-29 03:27:33 EST 2023-06-29  
3     2023-05-01 13:44:37 EST 2023-06-29  
4     2023-04-29 07:33:25 EST 2023-06-29  
...      

...

                                 name channel  status  count  \
0              526 rto-JULY22-CLK-hzb     sms  active    895   
1            ALL HZB_RHD.CC_all codes     sms  active  34102   
2      Copy HZB October 2022 Clickers     sms  active      0   
3                 Copy MBC_PN.FC_30DC     sms  active  16335   
4                             DSS ALL     sms  active  59207   
...                               ...     ...     ...    ...   
74444                  RTO-JULY22-CLK     sms  active   7579   
74445                 test-2565371505     sms  active   4255   
74446    viewrenttoownhomes URL check     sms  active   6386   
74447                        ZP461235     sms  active  85116   
74448                   ZP Last Month     sms  active   4110   

                  last_counted       Date  
0      2024-04-05 08:34:35 EST 2024-04-05  
1      2024-04-05 09:23:25 EST 2024-04-05  
2      2024-04-05 09:02:35 EST 2024-04-05  
3      2024-04-05 09:28:13 EST 2024-04-05  
4      2024

                                 name channel  status  count  \
0              526 rto-JULY22-CLK-hzb     sms  active    892   
1                      A4F_I.A4F_15DC     sms  active    619   
2                       A4F_I.A4F_7DC     sms  active    619   
3            ALL HZB_RHD.CC_all codes     sms  active  34136   
4      Copy HZB October 2022 Clickers     sms  active      0   
...                               ...     ...     ...    ...   
78903                  RTO-JULY22-CLK     sms  active   7579   
78904                 test-2565371505     sms  active   4255   
78905    viewrenttoownhomes URL check     sms  active   6386   
78906                        ZP461235     sms  active  85116   
78907                   ZP Last Month     sms  active   4110   

                  last_counted       Date  
0      2024-04-22 05:11:45 EST 2024-04-22  
1      2024-04-22 07:23:55 EST 2024-04-22  
2      2024-04-22 07:24:38 EST 2024-04-22  
3      2024-04-22 00:15:52 EST 2024-04-22  
4      2024

                                 name channel  status  count  \
0              526 rto-JULY22-CLK-hzb     sms  active    885   
1                      A4F_I.A4F_15DC     sms  active    184   
2                       A4F_I.A4F_7DC     sms  active     37   
3            ALL HZB_RHD.CC_all codes     sms  active  33880   
4      Copy HZB October 2022 Clickers     sms  active      0   
...                               ...     ...     ...    ...   
83641                  RTO-JULY22-CLK     sms  active   7579   
83642                 test-2565371505     sms  active   4255   
83643    viewrenttoownhomes URL check     sms  active   6386   
83644                        ZP461235     sms  active  85116   
83645                   ZP Last Month     sms  active   4110   

                  last_counted       Date  
0      2024-05-06 04:10:43 EST 2024-05-06  
1      2024-05-06 08:59:56 EST 2024-05-06  
2      2024-05-06 09:00:56 EST 2024-05-06  
3      2024-05-05 18:23:12 EST 2024-05-06  
4      2024

                               name channel  status  count  \
0            526 rto-JULY22-CLK-hzb     sms  active    895   
1                    A4F_I.A4F_15DC     sms  active     35   
2                     A4F_I.A4F_7DC     sms  active      0   
3          ALL HZB_RHD.CC_all codes     sms  active  33436   
4          Copy Copy MBC_PN.FC_30DC     sms  active   9318   
...                             ...     ...     ...    ...   
88378                RTO-JULY22-CLK     sms  active   7579   
88379               test-2565371505     sms  active   4255   
88380  viewrenttoownhomes URL check     sms  active   6386   
88381                      ZP461235     sms  active  85116   
88382                 ZP Last Month     sms  active   4110   

                  last_counted       Date  
0      2024-05-16 23:58:24 EST 2024-05-17  
1      2024-05-17 04:48:56 EST 2024-05-17  
2      2024-05-17 04:49:53 EST 2024-05-17  
3      2024-05-16 00:20:02 EST 2024-05-17  
4      2024-05-16 12:59:43 EST 2024

                               name channel  status  count  \
0            526 rto-JULY22-CLK-hzb     sms  active    906   
1                    A4F_I.A4F_15DC     sms  active      0   
2                     A4F_I.A4F_7DC     sms  active      0   
3          ALL HZB_RHD.CC_all codes     sms  active  32560   
4          Copy Copy MBC_PN.FC_30DC     sms  active   8449   
...                             ...     ...     ...    ...   
93239                RTO-JULY22-CLK     sms  active   7579   
93240               test-2565371505     sms  active   4255   
93241  viewrenttoownhomes URL check     sms  active   6386   
93242                      ZP461235     sms  active  85116   
93243                 ZP Last Month     sms  active   4110   

                  last_counted       Date  
0      2024-05-31 08:24:04 EST 2024-05-31  
1      2024-05-31 09:46:00 EST 2024-05-31  
2      2024-05-31 09:45:50 EST 2024-05-31  
3      2024-05-31 05:47:01 EST 2024-05-31  
4      2024-05-31 07:54:20 EST 2024

## Update Post Data <a class="anchor" id="chapter4"></a>

In [28]:
# Output dailt post data
legacy_post = pd.read_csv(output_filepath+'SMSposts.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_post.to_csv(input_filepath+'SMSposts_backup.csv',index=False)
# This is the creative stat raw data downloaded by the outsource 
updated_post = pd.read_csv(input_filepath+'SMSPosts.csv',index_col=False)
drop = updated_post['Pub Name'].str.contains('SMS', case = False, na = False)
updated_post = updated_post.loc[drop]
today = date.today()
first_date_of_month = today.replace(day=1)
updated_post = updated_post.loc[updated_post['Pub Id'].isna() == False]
current_year = datetime.now().year
updated_post['Dates'] = pd.to_datetime(updated_post['Dates'] + f'/{current_year}', format="%m/%d/%Y")#.apply(lambda x: x.replace(year=current_year))
updated_post['Dates'] = pd.to_datetime(updated_post['Dates']).dt.strftime('%m/%d/%Y')
updated_post = updated_post[updated_post['Dates']<=yesterday]
#updated_post['Dates'] = pd.to_datetime(updated_post['Dates']).dt.strftime('%m/%d/%Y')
legacy_post['Dates'] = pd.to_datetime(legacy_post['Dates'] )
min_date = updated_post['Dates'].min()
legacy_post = legacy_post[legacy_post['Dates']<min_date]
legacy_post['Dates'] = pd.to_datetime(legacy_post['Dates']).dt.strftime('%m/%d/%Y')
post = [legacy_post, updated_post]
output_post= pd.concat(post, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
print(output_post)
output_post['Pub Id'] = output_post['Pub Id'].astype(str)
output_post['Pub Name'] = output_post['Pub Name'].astype(str)
output_post = output_post.loc[output_post['Pub Name'].str.contains('SMS', case = False, na = False)]
output_post['Dates'] =  pd.to_datetime(output_post['Dates']) 
output_post.to_csv(output_filepath+'SMSposts.csv', index = False)

            Dates    Pub Id                           Pub Name  Raw Posts  \
0      01/01/2022  461227.0       LM_SMS_Edemographic_247Loans          0   
1      01/01/2022  461313.0  LM_SMS_NorthPointDirect_RentToOwn          0   
2      01/01/2022  460921.0            LM_SMS_Spark_CreditCard      17102   
3      01/01/2022  461147.0                 461147_ARM.SYF_SMS      15884   
4      01/01/2022  460917.0                   460917_JM.OP_SMS       9999   
...           ...       ...                                ...        ...   
39650  06/11/2024  461578.0                   461578.I.HUD_SMS         15   
39651  06/11/2024  461623.0          I_SMS_RXMG_ReadyRentToOwn         29   
39652  06/11/2024  461241.0                   461241_I.HIE_SMS          0   
39653  06/11/2024  460924.0                    460924_I.FB_SMS          2   
39654  06/11/2024  461348.0                   461348_I.CHQ_SMS          1   

       GeoIP Good  % IP Good  GeoIP Bad  % IP Bad  Goodhyg  % Good  ...  \


## Update Optout <a class="anchor" id="chapter5"></a>

In [29]:
from os import listdir
from os.path import isfile, join
import datetime as dt
                                                                     
smsreport_files = [f for f in listdir(smsreport_filepath) if isfile(join(smsreport_filepath, f)) & ('SMS_SC_SS_Report' in f)]
""" 
most_recent_smsreport_date = max([dt.datetime.strptime((date.split('.csv')[0][-8:]), "%Y%m%d") for date in smsreport_files])
smsreport_year = most_recent_smsreport_date.year
smsreport_month = most_recent_smsreport_date.month
segement_day = most_recent_smsreport_date.day

most_recent_smsreport_file = "SMS_SC_SS_Report{0}{1:0>2}{2:0>2}.csv".format(smsreport_year,smsreport_month,segement_day)

smsreport_report = pd.read_csv( smsreport_filepath+ most_recent_smsreport_file)

smsreport_report['Date'] = pd.to_datetime(most_recent_smsreport_date)
smsreport_report['Shortcode Name'] = ''
smsreport_report.loc[smsreport_report['Shortcode'].str.contains('FLC'),'Shortcode Name' ] ='FLC'
smsreport_report.loc[smsreport_report['Shortcode'].str.contains('HZB'),'Shortcode Name' ] ='HZB'
smsreport_report.loc[smsreport_report['Shortcode'].str.contains('MBC'),'Shortcode Name' ] ='MBC'
smsreport_report.loc[smsreport_report['Shortcode'].str.contains('CSS'),'Shortcode Name' ] ='CSS'
legacy_smsreport = pd.read_csv(output_filepath+'SMS_SC_SS_Report.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_smsreport.to_csv(input_filepath+'SMS_SC_SS_Report_backup.csv', index = False)
# This is the job stat raw data downloaded by the outsource 
legacy_smsreport['Date'] = pd.to_datetime(legacy_smsreport['Date'])
min_date = smsreport_report['Date'].min()
legacy_smsreport = legacy_smsreport[legacy_smsreport['Date']<min_date]
full_smsreport = [legacy_smsreport, smsreport_report]
output_smsreport = pd.concat(full_smsreport, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
print(output_smsreport)
output_smsreport.to_csv(output_filepath+'SMS_SC_SS_Report.csv', index = False)
"""
df = pd.DataFrame()
for i in smsreport_files:
    smsreport_report = pd.read_csv(smsreport_filepath+i)
    most_recent_smsreport_date = dt.datetime.strptime((i.split('.csv')[0][-8:]), "%Y%m%d")
    smsreport_report['Date'] = pd.to_datetime(most_recent_smsreport_date)
    smsreport_report['Shortcode Name'] = ''
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('Dollar Sensei'),'Shortcode Name' ] ='DSS'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('FLC'),'Shortcode Name' ] ='FLC'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('HZB'),'Shortcode Name' ] ='HZB'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('MBC'),'Shortcode Name' ] ='MBC'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('CSS'),'Shortcode Name' ] ='CSS'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('SVT'),'Shortcode Name' ] ='SVT'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('UAA'),'Shortcode Name' ] ='UAA'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('PRC'),'Shortcode Name' ] ='PRC'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('UAATF'),'Shortcode Name' ] ='UAATF'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('Aid4Famlilies'),'Shortcode Name' ] ='A4FLC'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('18332686782'),'Shortcode Name' ] ='MFATF'
    #added
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('18332685688'),'Shortcode Name' ] ='N3GTF'
    smsreport_report.loc[smsreport_report['Shortcode'].str.contains('18338987652'),'Shortcode Name' ] ='FBHTF'
    df = pd.concat([df,smsreport_report])
df.to_csv(output_filepath+'SMS_SC_SS_Report.csv', index = False)


# update apikey <a class="anchor" id="chapter6"></a>

In [30]:
from os import listdir
from os.path import isfile, join
import datetime as dt
                                                                     
smsreport_files = [f for f in listdir(smsapikey_filepath) if isfile(join(smsapikey_filepath, f)) & ('apikeyreport' in f)]

df = pd.DataFrame()
for i in smsreport_files:
    smsapikey_report = pd.read_csv(smsapikey_filepath+i)
    smsapikey_report['Date'] = pd.to_datetime(smsapikey_report['Date'])
    smsapikey_report['DP.SV'] = smsapikey_report['Apikey'].str.split(" |-",expand = True)[0]
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('I.FD',na= False),'DP.SV' ] ='I.FD'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('I.RTOD',na= False),'DP.SV' ] ='I.RTOD'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('I.LD',na= False),'DP.SV' ] ='I.LD'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('I.NFD',na= False),'DP.SV' ] ='I.NFD'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('A4F',na= False),'DP.SV' ] ='I.A4F'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('MFA',na= False),'DP.SV' ] ='I.MFA'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('N3G',na= False),'DP.SV' ] ='I.N3G'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('FBH',na= False),'DP.SV' ] ='I.FBH'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('THM',na= False),'DP.SV' ] ='I.THM'
    smsapikey_report.loc[smsapikey_report['Apikey'].str.contains('FRH',na= False),'DP.SV' ] ='I.FRH'
    df = pd.concat([df,smsapikey_report])
df = df.drop_duplicates()
df.to_csv(output_filepath+'SMS_SC_SS_Apikey.csv', index = False)


## Update Flow <a class="anchor" id="chapter7"></a>

In [31]:
legacy_flows = pd.read_csv(output_filepath+'SMS_SC_SS_Flows.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_flows.to_csv(input_filepath+'SMS_SC_SS_Flows_backup.csv',index=False)
# This is the creative stat raw data downloaded by the outsource 
updated_flows = pd.read_csv(input_filepath+'SMS_SC_SS_Flows.csv',index_col=False)
legacy_flows['Period'] = pd.to_datetime(legacy_flows['Period'],format='mixed')
min_date = updated_flows['Period'].min()
legacy_flows = legacy_flows[legacy_flows['Period']<min_date]
flows = [legacy_flows, updated_flows]
output_flows = pd.concat(flows, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
print(output_flows)
output_flows.to_csv(output_filepath+'SMS_SC_SS_Flows.csv', index = False)


         Id                    Name               Period  \
0      5009           PA.SWP-MBC-PR  2023-03-06 00:00:00   
1      5009           PA.SWP-MBC-PR  2023-03-07 00:00:00   
2      5009           PA.SWP-MBC-PR  2023-03-07 00:00:00   
3      5009           PA.SWP-MBC-PR  2023-03-08 00:00:00   
4      5009           PA.SWP-MBC-PR  2023-03-08 00:00:00   
...     ...                     ...                  ...   
52810  5154  AL.PL.4-HZB-VZN-BU-SFW           2024-06-11   
52811  5154  AL.PL.4-HZB-VZN-BU-SFW           2024-06-11   
52812  5154  AL.PL.4-HZB-VZN-BU-SFW           2024-06-11   
52813  5154  AL.PL.4-HZB-VZN-BU-SFW           2024-06-11   
52814  5154  AL.PL.4-HZB-VZN-BU-SFW           2024-06-11   

                                          Shortcode                ListName  \
0      MyBenefitsClub :: MBC :: 80837 [80837 : 306]  MyBenefitsClub [12421]   
1      MyBenefitsClub :: MBC :: 80837 [80837 : 306]  MyBenefitsClub [12421]   
2      MyBenefitsClub :: MBC :: 80837 [808

## Update Creatives Data <a class="anchor" id="chapter8"></a>

In [32]:
legacy_creatives = pd.read_csv(output_filepath+'SMS_SC_SS_CreativesStats.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_creatives.to_csv(input_filepath+'SMS_SC_SS_CreativesStats_backup.csv',index=False)
# This is the creative stat raw data downloaded by the outsource 
updated_creatives = pd.read_csv(input_filepath+'creativestats.csv',index_col=False)
legacy_creatives['Tstamp'] = pd.to_datetime(legacy_creatives['Tstamp'],format='mixed')
min_date = updated_creatives['Tstamp'].min()
legacy_creatives = legacy_creatives[legacy_creatives['Tstamp']<min_date]
creativestats = [legacy_creatives, updated_creatives]
output_creatives = pd.concat(creativestats, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
print(output_creatives)
output_creatives['Tstamp'] = pd.to_datetime(output_creatives['Tstamp'],format='mixed')
output_creatives.to_csv(output_filepath+'SMS_SC_SS_CreativesStats.csv', index = False)


        JobId                     Tstamp                              Offer  \
0           0  2021-10-01 00:00:00+00:00                                NaN   
1      237891  2021-10-01 17:00:00+00:00  B2USRA - Cheris Studio Benefits A   
2           0  2021-10-02 00:00:00+00:00                                NaN   
3      238383  2021-10-02 17:00:00+00:00  B2USRA - Cheris Studio Benefits A   
4           0  2021-10-03 00:00:00+00:00                                NaN   
...       ...                        ...                                ...   
48478  672692       2024-06-11T17:00:59Z                        13113 - DSS   
48479  672557       2024-06-11T18:00:36Z                    13059 - N3G DIR   
48480  672557       2024-06-11T18:00:36Z                    13059 - N3G DIR   
48481  672556       2024-06-11T19:00:05Z                    12610 - MFA DIR   
48482  672556       2024-06-11T19:00:05Z                    12610 - MFA DIR   

                     Segment  CreativeId         Cr

In [33]:
output_creatives['Tstamp'].astype(str).str[:10].unique()

array(['2021-10-01', '2021-10-02', '2021-10-03', '2021-10-04',
       '2021-10-05', '2021-10-06', '2021-10-07', '2021-10-08',
       '2021-10-09', '2021-10-10', '2021-10-11', '2021-10-12',
       '2021-10-13', '2021-10-14', '2021-10-15', '2021-10-16',
       '2021-10-17', '2021-10-18', '2021-10-19', '2021-10-20',
       '2021-10-21', '2021-10-22', '2021-10-23', '2021-10-24',
       '2021-10-25', '2021-10-26', '2021-10-27', '2021-10-28',
       '2021-10-29', '2021-10-30', '2021-10-31', '2021-11-01',
       '2021-11-02', '2021-11-03', '2021-11-04', '2021-11-05',
       '2021-11-06', '2021-11-07', '2021-11-08', '2021-11-09',
       '2021-11-10', '2021-11-11', '2021-11-12', '2021-11-13',
       '2021-11-14', '2021-11-15', '2021-11-16', '2021-11-17',
       '2021-11-18', '2021-11-19', '2021-11-20', '2021-11-21',
       '2021-11-22', '2021-11-23', '2021-11-24', '2021-11-25',
       '2021-11-26', '2021-11-27', '2021-11-28', '2021-11-29',
       '2021-11-30', '2021-12-01', '2021-12-02', '2021-

## Update Jobs <a class="anchor" id="chapter9"></a>

In [34]:
legacy_jobs = pd.read_csv(output_filepath+'SMS_SC_SS_Jobs.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong 
legacy_jobs.to_csv(input_filepath+'SMS_SC_SS_Jobs_backup.csv', index = False)
# This is the job stat raw data downloaded by the outsource 
updated_jobs = pd.read_csv(input_filepath+'jobs.csv',index_col=False)
legacy_jobs['scheduled_tstamp'] = pd.to_datetime(legacy_jobs['scheduled_tstamp'],format = "mixed")
min_date = updated_jobs['scheduled_tstamp'].min()
legacy_jobs = legacy_jobs[legacy_jobs['scheduled_tstamp']<min_date]
jobs = [legacy_jobs, updated_jobs]
output_jobs = pd.concat(jobs, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
print(output_jobs)
output_jobs.to_csv(output_filepath+'SMS_SC_SS_Jobs.csv', index = False)

           id                                       name  \
0      237891                      B2USRA_1 [2021-10-01]   
1      238383                      B2USRA_1 [2021-10-02]   
2      238832                      B2USRA_1 [2021-10-03]   
3      239249                      B2USRA_1 [2021-10-04]   
4      239695                      B2USRA_1 [2021-10-05]   
...       ...                                        ...   
38363  672685    SS_HZB_AL-PL-2-30DC-VZN_13113_P_11Jun24   
38364  672689   SS_HZB_AL-PL-2-30DC-VZN_12588_CT_11Jun24   
38365  672692   SS_DSS_JET-ZTA-1PLD-21DC_13113_P_11Jun24   
38366  672695  SS_DSS_JET-ZTA-1PLD-21DC_12588_CT_11Jun24   
38367  673109        SS_SVT_AL-PL-4-30DC_13194_P_11Jun24   

                                   offer  shortcode             start_tstamp  \
0      B2USRA - Cheris Studio Benefits A      70610  2021-10-01 13:00:09 EST   
1      B2USRA - Cheris Studio Benefits A      70610  2021-10-02 13:00:10 EST   
2      B2USRA - Cheris Studio Benefits 

In [35]:
updated_jobs

,id,name,offer,shortcode,start_tstamp,end_tstamp,scheduled_tstamp,status_text,remote_status,segments,...,sent,delivered,not_delivered,optout,clicks,conversions,revenue,cost,ecpm,roi
0,672473,SS_MBC_I-RC-15DC_13059_P_11Jun24,13059 - MBC,80837,2024-06-11 11:31:13 EST,2024-06-11 11:39:57 EST,2024-06-11T15:30:50Z,Sending Complete,NaN,MBC_I.RC_15DC,...,5244,5240,21,99,332,0,0.0,7.8660,0.0,-7.8660
1,672474,SS_MBC_I-RC-15DC_13070_CT_11Jun24,13070 - MBC,80837,2024-06-11 11:30:37 EST,2024-06-11 11:36:24 EST,2024-06-11T15:30:21Z,Sending Complete,NaN,MBC_I.RC_15DC,...,2793,2793,10,20,132,0,0.0,4.1895,0.0,-4.1895
2,672475,SS_FLC_I-RC-30DC-VZN-MAY24_13362_P_11Jun24,13362 - FLC,51797,2024-06-11 13:00:24 EST,2024-06-11 13:13:04 EST,2024-06-11T17:00:00Z,Sending Complete,NaN,FLC_I.RC_30DC_VZN_MAY24,...,2945,2945,92,4,315,0,0.0,4.4175,0.0,-4.4175
3,672476,SS_DSS_SM-SRV-30DC_13348_P_11Jun24,13348 - DSS,31232,2024-06-11 13:00:47 EST,2024-06-11 13:20:13 EST,2024-06-11T17:00:09Z,Sending Complete,NaN,DSS_SM.SRV_30DC,...,6884,6634,25,60,266,0,0.0,10.3260,0.0,-10.3260
4,672477,SS_HZB_SM-SRV-30DC-VZN_13362_OT_11Jun24,13362 - HZB,44345,2024-06-11 13:01:48 EST,2024-06-11 13:10:52 EST,2024-06-11T17:00:55Z,Sending Complete,NaN,HZB_SM.SRV_30DC_VZN,...,2735,2658,164,25,236,0,0.0,19.1450,0.0,-19.1450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,672685,SS_HZB_AL-PL-2-30DC-VZN_13113_P_11Jun24,13113 - HZB,44345,2024-06-11 11:15:53 EST,2024-06-11 11:18:54 EST,2024-06-11T15:15:16Z,Sending Complete,NaN,HZB_AL.PL.2_30DC_VZN,...,3400,3276,86,41,62,0,0.0,47.6000,0.0,-47.6000
63,672689,SS_HZB_AL-PL-2-30DC-VZN_12588_CT_11Jun24,12588 - EPCVIP MLUSA HZB,44345,2024-06-11 11:15:53 EST,2024-06-11 11:17:25 EST,2024-06-11T15:15:34Z,Sending Complete,NaN,HZB_AL.PL.2_30DC_VZN,...,1200,1146,36,2,99,0,0.0,14.2660,0.0,-14.2660
64,672692,SS_DSS_JET-ZTA-1PLD-21DC_13113_P_11Jun24,13113 - DSS,31232,2024-06-11 13:02:07 EST,2024-06-11 13:08:44 EST,2024-06-11T17:00:59Z,Sending Complete,NaN,DSS_JET.ZTA_1PLD.21DC,...,587,551,3,6,22,0,0.0,0.8805,0.0,-0.8805
65,672695,SS_DSS_JET-ZTA-1PLD-21DC_12588_CT_11Jun24,12588 - EPCVIP MLUSA DSS,31232,2024-06-11 13:01:07 EST,2024-06-11 13:14:26 EST,2024-06-11T17:00:26Z,Sending Complete,NaN,DSS_JET.ZTA_1PLD.21DC,...,2198,2000,15,6,136,0,0.0,3.2970,0.0,-3.2970


In [36]:
output_jobs['start_tstamp'].str[:10].unique()

array(['2021-10-01', '2021-10-02', '2021-10-03', '2021-10-04',
       '2021-10-05', '2021-10-06', '2021-10-07', '2021-10-08',
       '2021-10-13', '2021-10-14', '2021-10-15', '2021-10-16',
       '2021-10-17', '2021-10-18', '2021-10-19', '2021-10-20',
       '2021-10-21', '2021-10-22', '2021-10-23', '2021-10-24',
       '2021-10-25', '2021-10-26', '2021-10-27', '2021-10-28',
       '2021-10-29', '2021-10-30', '2021-10-31', '2021-11-01',
       '2021-11-02', '2021-11-03', '2021-11-04', '2021-11-05',
       '2021-11-06', '2021-11-07', '2021-11-08', '2021-11-09',
       '2021-11-10', '2021-11-11', '2021-11-12', '2021-11-15',
       '2021-11-16', '2021-11-17', '2021-11-18', '2021-11-19',
       '2021-11-20', '2021-11-21', '2021-11-22', '2021-11-23',
       '2021-11-24', '2021-11-25', '2021-11-26', '2021-11-27',
       '2021-11-28', '2021-11-29', '2021-11-30', '2021-12-01',
       '2021-12-02', '2021-12-03', '2021-12-04', '2021-12-05',
       '2021-12-06', '2021-12-07', '2021-12-08', '2021-

## Update Post Data

In [37]:
# # Output dailt post data
# legacy_post = pd.read_csv(output_filepath+'SMSposts.csv',index_col=False)
# # backup previous data file to in case we can revert back if there's something wrong 
# legacy_post.to_csv(input_filepath+'SMSposts_backup.csv',index=False)
# # This is the creative stat raw data downloaded by the outsource 
# updated_post = pd.read_csv(input_filepath+'SMSPosts.csv',index_col=False)
# drop = updated_post['Pub Name'].str.contains('SMS', case = False, na = False)
# updated_post = updated_post.loc[drop]
# today = date.today()
# first_date_of_month = today.replace(day=1)
# updated_post = updated_post.loc[updated_post['Pub Id'].isna() == False]
# current_year = datetime.now().year
# updated_post['Dates'] = pd.to_datetime(updated_post['Dates']+ f'/{current_year}', format="%m/%d/%Y")#.apply(lambda x: x.replace(year=current_year))
# updated_post['Dates'] = pd.to_datetime(updated_post['Dates']).dt.strftime('%m/%d/%Y')
# updated_post = updated_post[updated_post['Dates']<=yesterday]
# #updated_post['Dates'] = pd.to_datetime(updated_post['Dates']).dt.strftime('%m/%d/%Y')
# legacy_post['Dates'] = pd.to_datetime(legacy_post['Dates'] )
# min_date = updated_post['Dates'].min()
# legacy_post = legacy_post[legacy_post['Dates']<min_date]
# legacy_post['Dates'] = pd.to_datetime(legacy_post['Dates']).dt.strftime('%m/%d/%Y')
# post = [legacy_post, updated_post]
# output_post= pd.concat(post, axis = 0, join = "outer",ignore_index = True, keys = None, levels = None, names = None, verify_integrity = False, copy = True)
# print(output_post)
# output_post['Pub Id'] = output_post['Pub Id'].astype(str)
# output_post['Pub Name'] = output_post['Pub Name'].astype(str)
# output_post = output_post.loc[output_post['Pub Name'].str.contains('SMS', case = False, na = False)]
# output_post['Dates'] =  pd.to_datetime(output_post['Dates']) 
# output_post.to_csv(output_filepath+'SMSposts.csv', index = False)

In [38]:
output_post['Dates'].unique()

<DatetimeArray>
['2022-01-01 00:00:00', '2022-01-02 00:00:00', '2022-01-03 00:00:00',
 '2022-01-04 00:00:00', '2022-01-05 00:00:00', '2022-01-06 00:00:00',
 '2022-01-07 00:00:00', '2022-01-08 00:00:00', '2022-01-09 00:00:00',
 '2022-01-10 00:00:00',
 ...
 '2024-06-02 00:00:00', '2024-06-03 00:00:00', '2024-06-04 00:00:00',
 '2024-06-05 00:00:00', '2024-06-06 00:00:00', '2024-06-07 00:00:00',
 '2024-06-08 00:00:00', '2024-06-09 00:00:00', '2024-06-10 00:00:00',
 '2024-06-11 00:00:00']
Length: 891, dtype: datetime64[ns]

## Update Offers <a class="anchor" id="chapter10"></a>

In [39]:
offers_old = pd.read_csv(output_filepath + 'SMS_SC_SS_Offers.csv',index_col=False)
# backup previous data file to in case we can revert back if there's something wrong
offers_old.to_csv(input_filepath+'SMS_SC_SS_Offers_backup.csv', index = False)
# This is the offer raw data downloaded by the outsource 
offers = pd.read_csv(input_filepath+'SMS_SC_SS_Offers.csv',index_col=False)
offers['hitpath_offer_id'] = offers['name'].str.split(' ',expand = True)[0].str.strip()
offers = offers[['id', 'name','hitpath_offer_id', 'type', 'status', 'redirect_type', 'conversion_event',
       'conversion_payout', 'currency']]
print(offers.head(5))
offers.to_csv(output_filepath+'SMS_SC_SS_Offers.csv', index = False)

      id                name hitpath_offer_id type  status redirect_type  \
0  20267   6963 - WM UBG CSS             6963  sms  active          HTTP   
1  40816   12088 - BE OE HZB            12088  sms  active          HTTP   
2  48116  12610 - AVL CL SVT            12610  sms  active          HTTP   
3  48117  12611 - AVL QL HZB            12611  sms  active          HTTP   
4  40935   12163 - BK OD HZB            12163  sms  active          HTTP   

  conversion_event  conversion_payout currency  
0              CPA                  0      USD  
1              CPA                  0      USD  
2              CPA                  0      USD  
3              CPA                  0      USD  
4              CPA                  0      USD  


In [40]:
end_time = time.time()

In [41]:
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time:.2f} seconds")

Execution time: 1446.02 seconds
